In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

#from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
# for keras > 2.0, use the following code

from keras.utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')


In [18]:
import tensorflow as tf
gpus= tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0],'GPU')

In [19]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

In [20]:
#Basic Exploration
print(train_data.columns)
print(train_data.shape)
print(test_data.columns)
print(test_data.shape)
print(train_data.head)


Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)
(42000, 785)
Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)
(28000, 784)
<bound method NDFrame.head of        label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0 

In [21]:
# Extracting features and target column
X = train_data.drop('label', axis = 1)
y = train_data['label']


In [22]:
# Step 2: Data Preprocessing

X = X.values

# Reshape the input images to the required format (assuming channels last)
X = X.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode the labels
y = to_categorical(y, 10)

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))

# Fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 classes for digits 0-9

In [24]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
657/657 [==============================] - 7s 10ms/step - loss: 0.4997 - accuracy: 0.8626
Epoch 2/10
657/657 [==============================] - 6s 10ms/step - loss: 0.1720 - accuracy: 0.9479
Epoch 3/10
657/657 [==============================] - 7s 10ms/step - loss: 0.5357 - accuracy: 0.8435
Epoch 4/10
657/657 [==============================] - 6s 10ms/step - loss: 0.2373 - accuracy: 0.9286
Epoch 5/10
657/657 [==============================] - 6s 10ms/step - loss: 0.1629 - accuracy: 0.9487
Epoch 6/10
657/657 [==============================] - 6s 10ms/step - loss: 0.1315 - accuracy: 0.9591
Epoch 7/10
657/657 [==============================] - 6s 10ms/step - loss: 0.1115 - accuracy: 0.9646
Epoch 8/10
657/657 [==============================] - 6s 10ms/step - loss: 0.0955 - accuracy: 0.9702
Epoch 9/10
657/657 [==============================] - 6s 10ms/step - loss: 0.0845 - accuracy: 0.9740
Epoch 10/10
657/657 [==============================] - 6s 10ms/step - loss: 0.4697 - accura

In [26]:
X_test = test_data
X_test = X_test.values

In [27]:
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [28]:
predictions = model.predict(X_test)

875/875 [==============================] - 1s 1ms/step


In [29]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = predicted_classes.reshape(-1, 1)

In [30]:
imageIds = [i for i in range(1,len(predictions)+1)]
df = pd.DataFrame()
df.insert(0, 'ImageId', imageIds)
df.insert(1, 'Label', predicted_classes)

In [31]:
df.to_csv('predictions.csv', index=False)
